# Rate performance

   * [Comparison new vs old EG ID](#Comparisono-baseline-vs-Manos-EGID)




In [1]:
import sys
sys.path.insert(0, '..')
# sys.path.insert(1, '../python/')
sys.path.insert(1, 'python/')

import ROOT
import root_numpy as rnp
# import python.l1THistos as histos
import math
import uuid
import pandas as pd

from drawingTools import *

# ROOT.enableJSVis()
ROOT.enableJSVis()

#from drawingTools import *

normalized_histos = list()

Welcome to JupyROOT 6.14/02


In [2]:
# %load python/drawingTools.py


In [3]:
# %load samples.py
import ROOT
import pandas as pd

version = 'v101'

version_TDR = 'v121'

version_V11 = 'v144A'
version_NM = 'v142A'
version_V9 = 'v121'


# version_V8 = version
version_V9 = 'v132B'
version_V10 = 'v133B'

# -------------------------------------------------------------------------

samples_ele = [
    Sample('ele_flat2to100_PU0', 'PU0', version, 'ele'),
    Sample('ele_flat2to100_PU200', 'PU200', version, 'ele')
    ]

samples_ele_V8 = [
    Sample('ele_flat2to100_PU0', 'PU0', version, 'V8'),
    Sample('ele_flat2to100_PU200', 'PU200', version, 'V8')
    ]


samples_ele_V9 = [
    Sample('all_flat5to80_PU0', 'PU0', 'v75t', 'V9'),
    Sample('all_flat5to80_PU200', 'PU200', 'v75t', 'V9')
    ]

samples_photons = [
    Sample('photon_flat8to150_PU0', 'PU0', version, 'photon'),
    Sample('photon_flat8to150_PU200', 'PU200', version, 'photon')
    ]

samples_pions = [
    Sample('pion_flat2to100_PU0', 'PU0', version, 'pions'),
    ]

# samples_nugus = [
#     Sample('nugun_alleta_pu0', 'PU0', version, 'mb'),
#     Sample('nugun_alleta_pu200', 'PU200', version, 'mb')
#     ]


samples_nugunrates_TDR = [
    Sample('nugun_alleta_pu200_rate', 'PU200', version_TDR, 'TDR')
    ]


samples_nugunrates_V9 = [
    Sample('nugun_alleta_pu200_rate', 'PU200', version_V9, 'V9')
    ]

samples_nugunrates_V10 = [
    Sample('nugun_alleta_pu200_rate', 'PU200', version_V10, 'V10')
    ]

samples_nugunrates_V11 = [
    Sample('nugun_alleta_pu200_rate', 'PU200', version_V11, 'V11')
    ]

samples_nugunrates_NM = [
    Sample('nugun_alleta_pu200_rate', 'PU200', version_NM, 'NM')
    ]







In [4]:
# %load python/settings.py

# === samples =====================================================
import pprint
import python.plotters_config as plotters



samples = []

# samples += samples_nugunrates
# samples += samples_nugunrates_V8
samples += samples_nugunrates_V11
samples += samples_nugunrates_NM

# samples += samples_nugunrates_V9
# samples += samples_nugunrates_V10

samples += samples_nugunrates_TDR

for smp in samples:
    smp.open_file()


sample = 'V10'

# === TP ==========================================================
tps = [
       'EG',
       'EGBRL',
#        'TkEle',
      'TkEleEL',
       'TkEleBRL',
#       'TkEleELBRL',
#        'TkEleALL',
#       'TkEleELALL',
       'tkEleEE',
       'tkEleEB',
       'tkEleEllAll'

]

# === Load the Histo Primitives ====================================
histo_primitives = samples[-1].build_file_primitive_index()
print histo_primitives

# print histo_primitives.data.unique()
# === TP selections ================================================
tp_select = {}

for tp in tps:
    tp_select[tp] = histo_primitives[histo_primitives.tp == tp].tp_sel.unique().tolist()

# ==== GEN selections ===============================================
gen_select ={}
for tp in tps:
    gen_select[tp] = histo_primitives[histo_primitives.tp == tp].gen_sel.unique().tolist()


import pprint
pp = pprint.PrettyPrinter(indent=4)
print '--- TPs: '
pp.pprint(tps)
print '--- TP selections:'
pp.pprint(tp_select)
print '--- GEN selections:'
pp.pprint(gen_select)


gen_part_selections: 11
--- RateHistos
# of plots: 372
# of primitives: 124
      classtype gen_sel          tp            tp_sel
0    RateHistos    None     TkEleEL         EGq4EtaBC
1    RateHistos    None  TkEleELBRL        EtaFIso0p1
2    RateHistos    None     TkEleEL    EGq4Iso0p2EtaD
3    RateHistos    None     TkEleEL         EGq2EtaDE
4    RateHistos    None     TkEleEL   EGq4Iso0p2EtaBC
5    RateHistos    None  TkEleELBRL               all
6    RateHistos    None     TkEleEL         EGq3EtaDE
7    RateHistos    None  TkEleELBRL         LooseTkID
8    RateHistos    None       EGBRL              EtaF
9    RateHistos    None       EGBRL     EtaFLooseTkID
10   RateHistos    None     TkEleEL  EGq3Iso0p3EtaBCD
11   RateHistos    None     TkEleEL   EGq3Iso0p2EtaBC
12   RateHistos    None     TkEleEL              EGq4
13   RateHistos    None     TkEleEL        EGq2EtaBCD
14   RateHistos    None     TkEleEL              EGq5
15   RateHistos    None     TkEleEL   EGq2Iso0p1EtaBC
16   R

In [9]:
ROOT.gStyle.SetPadGridX(True)
ROOT.gStyle.SetPadGridY(True)


In [10]:
import python.collections as collections
import python.selections as selections

labels_dict = {}

evm = collections.EventManager()
labels_dict.update(evm.get_labels())
selm = selections.SelectionManager()
labels_dict.update(selm.get_labels())



In [11]:
for tp in tps:
    print 'TP: {}'.format(tp)
    print histo_primitives[histo_primitives.tp == tp].tp_sel.unique()
    print histo_primitives[histo_primitives.tp == tp].gen_sel.unique()

TP: EG
['EGq2EtaD' 'EGq5EtaBC' 'EGq3EtaBCD' 'EGq2' 'EGq4' 'EGq3EtaBC' 'EGq5EtaDE'
 'EGq4EtaBC' 'EGq5EtaD' 'EGq2EtaBCD' 'EGq5EtaBCD' 'EGq3' 'EGq2EtaDE'
 'EGq4EtaD' 'EGq4EtaDE' 'EGq3EtaD' 'EGq4EtaBCD' 'EGq5' 'EGq2EtaBC'
 'EGq3EtaDE']
[None]
TP: EGBRL
['EtaF' 'EtaFLooseTkID' 'all' 'LooseTkID']
[None]
TP: TkEleEL
['EGq4EtaBC' 'EGq4Iso0p2EtaD' 'EGq2EtaDE' 'EGq4Iso0p2EtaBC' 'EGq3EtaDE'
 'EGq3Iso0p3EtaBCD' 'EGq3Iso0p2EtaBC' 'EGq4' 'EGq2EtaBCD' 'EGq5'
 'EGq2Iso0p1EtaBC' 'EGq5EtaD' 'EGq4Iso0p3EtaBC' 'EGq5Iso0p1EtaD' 'EGq4EtaD'
 'EGq2Iso0p3EtaBCD' 'EGq3EtaBC' 'EGq3Iso0p1EtaBC' 'EGq4Iso0p2EtaBCD'
 'EGq3Iso0p1EtaD' 'EGq3EtaBCD' 'EGq4Iso0p1EtaD' 'EGq5EtaDE'
 'EGq5Iso0p3EtaDE' 'EGq2Iso0p2EtaBC' 'EGq5Iso0p1' 'EGq2Iso0p3EtaD'
 'EGq5Iso0p2' 'EGq5Iso0p3EtaBC' 'EGq3Iso0p3EtaBC' 'EGq5Iso0p1EtaBCD'
 'EGq2Iso0p3' 'EGq2Iso0p1EtaDE' 'EGq3Iso0p1EtaBCD' 'EGq2Iso0p1'
 'EGq4Iso0p2EtaDE' 'EGq2Iso0p1EtaD' 'EGq3' 'EGq3Iso0p1EtaDE'
 'EGq2Iso0p3EtaDE' 'EGq2Iso0p1EtaBCD' 'EGq3Iso0p3EtaD' 'EGq2Iso0p2EtaDE'
 'EGq4EtaBCD'

In [12]:
histo_primitives[:10]

,classtype,gen_sel,tp,tp_sel
0,RateHistos,None,TkEleEL,EGq4EtaBC
1,RateHistos,None,TkEleELBRL,EtaFIso0p1
2,RateHistos,None,TkEleEL,EGq4Iso0p2EtaD
3,RateHistos,None,TkEleEL,EGq2EtaDE
4,RateHistos,None,TkEleEL,EGq4Iso0p2EtaBC
5,RateHistos,None,TkEleELBRL,all
6,RateHistos,None,TkEleEL,EGq3EtaDE
7,RateHistos,None,TkEleELBRL,LooseTkID
8,RateHistos,None,EGBRL,EtaF
9,RateHistos,None,EGBRL,EtaFLooseTkID


In [13]:
%%time

import python.l1THistos as histos

hplot = HPlot(samples, labels_dict)

hplot.create_histo_proxies(histos.RateHistos)


--- RateHistos
# of plots: 612
# of primitives: 204
--- RateHistos
# of plots: 612
# of primitives: 204
--- RateHistos
# of plots: 372
# of primitives: 124
CPU times: user 3.04 s, sys: 135 ms, total: 3.18 s
Wall time: 3.93 s


In [14]:
hplot.data['sample'].unique()

array(['V11', 'NM', 'TDR'], dtype=object)

In [15]:
hplot.data[:10]

,sample,pu,tp,tp_sel,gen_sel,classtype,histo
0,V11,PU200,EG,EGq5EtaB,None,python.l1THistos.RateHistos,<drawingTools.HProxy instance at 0x11f0fe050>
1,V11,PU200,tkEleEE,EGq2Iso0p2EtaBCD,None,python.l1THistos.RateHistos,<drawingTools.HProxy instance at 0x11ba9bea8>
2,V11,PU200,tkEleEE,EGq5Iso0p1EtaDE,None,python.l1THistos.RateHistos,<drawingTools.HProxy instance at 0x11ba9ba70>
3,V11,PU200,tkEleEE,EGq5,None,python.l1THistos.RateHistos,<drawingTools.HProxy instance at 0x11f0eab48>
4,V11,PU200,tkEleEE,EGq2EtaB,None,python.l1THistos.RateHistos,<drawingTools.HProxy instance at 0x11f106098>
5,V11,PU200,tkEleEE,EGq2Iso0p1EtaB,None,python.l1THistos.RateHistos,<drawingTools.HProxy instance at 0x10c24a950>
6,V11,PU200,EG,EGq4EtaBCDE,None,python.l1THistos.RateHistos,<drawingTools.HProxy instance at 0x11f0f0a70>
7,V11,PU200,tkEleEE,EGq3Iso0p3EtaBC,None,python.l1THistos.RateHistos,<drawingTools.HProxy instance at 0x11ba9be60>
8,V11,PU200,tkEleEE,EGq3Iso0p1EtaBCDE,None,python.l1THistos.RateHistos,<drawingTools.HProxy instance at 0x11f079fc8>
9,V11,PU200,EGBRL,EtaF,None,python.l1THistos.RateHistos,<drawingTools.HProxy instance at 0x11f0bf9e0>


In [16]:
hplot.data[hplot.data.tp == 'TkEleELALL'].tp_sel.unique()

array(['EtaBC', 'Iso0p1', 'EtaF', 'EtaFLooseTkID', 'EtaA', 'LooseTkID',
       'EtaFIso0p1', 'all'], dtype=object)

#### 3D clusters matched to GEN

In [17]:
# version_V9 = 'v113'
scaling_params =  pd.read_csv(r'scaling_params_{}.csv'.format(version_NM))

In [18]:
scaling_params

,Unnamed: 0,tp,tp_sel,params,sample,pu
0,0,EG,EGq5EtaBC,"(6.037856085779983, 1.0989328915838663)",ele-V9,PU0
1,1,TkEleEL,EGq5EtaBC,"(5.355004263517035, 1.1165288275760397)",ele-V9,PU0
2,2,EGBRL,allEtaF,"(4.33427086849217, 1.0092957507706983)",ele-V9,PU0
3,3,EGBRL,LooseTkIDEtaF,"(3.510329235810837, 1.0305315294454913)",ele-V9,PU0
4,4,TkEleELBRL,allEtaF,"(0.1749948001067856, 1.109074287128954)",ele-V9,PU0
5,5,EG,EGq5EtaBC,"(7.892955246439467, 1.0450598534917357)",ele-V9,PU200
6,6,TkEleEL,EGq5EtaBC,"(5.9020892979972714, 1.0955363006244943)",ele-V9,PU200
7,7,EGBRL,allEtaF,"(4.209194442902704, 0.9991760195740905)",ele-V9,PU200
8,8,EGBRL,LooseTkIDEtaF,"(6.734526207925173, 0.9312246788755905)",ele-V9,PU200
9,9,TkEleELBRL,allEtaF,"(2.5102979469553617, 1.0332268479075164)",ele-V9,PU200


In [19]:
# eval(scaling_params[(scaling_params.pu == 'PU200') & (scaling_params.tp == 'EG') & (scaling_params.tp_sel == 'EGq3EtaBC')].params.iloc[0])

In [20]:
def convertRateToGraph(histo_rate, params=None):
    xaxis = histo_rate.GetXaxis()
    fNpoints = xaxis.GetNbins()
    fX = [0]*fNpoints
    fEX = [0]*fNpoints
    fY = [0]*fNpoints
    fEY = [0]*fNpoints

    graph = ROOT.TGraphAsymmErrors()
    graph.Set(fNpoints)
    for i in range(0, fNpoints):
        if params is None:
            fX[i] = xaxis.GetBinLowEdge(i + 1)
        else:
            fX[i] = params[0] + params[1]*xaxis.GetBinLowEdge(i + 1)
        fY[i] = histo_rate.GetBinContent(i + 1)
        fEX[i] = histo_rate.GetBinWidth(i + 1) * ROOT.gStyle.GetErrorX()
        fEY[i] = histo_rate.GetBinError(i + 1)
    return fX, fY, fEX, fEX, fEY, fEY


In [ ]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, ['V10', 'V11','V9', 'TDR'], 'PU200', ['HMvDRshapeDrCalib', 'HMvDRshapeDrCalibNew'], 'EmEtaBC', None)
for his in hsets:
    his.normalize(31000)
draw([his.h_pt for his in hsets], labels, text=text,  
     min_y=1, max_y=40000,
     min_x=0, max_x=60,
     logy=True, x_axis_label='online p_{T} thresh. [GeV]')


In [71]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, ['V10', 'V11','V9', 'TDR'], 'PU200', ['HMvDR'], 'EmEtaBC', None)
for his in hsets:
    his.normalize(31000)
draw([his.h_pt for his in hsets], labels, text=text,  
     min_y=1, max_y=40000,
     min_x=0, max_x=60,
     logy=True, x_axis_label='online p_{T} thresh. [GeV]')


No match found for: sample: ['V10', 'V11', 'V9', 'TDR'] pu: PU200, tp: ['HMvDR'], tp_sel: EmEtaBC, gen_sel: None, classtype: python.l1THistos.RateHistos


TypeError: 'NoneType' object is not iterable

In [73]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, ['V11','NM', 'TDR'], 'PU200', ['EG'], 'EGq5EtaBC', None)
for his in hsets:
    his.normalize(31000)
draw([his.h_pt for his in hsets], labels, text=text,  
     min_y=1, max_y=40000,
     min_x=0, max_x=60,
     logy=True, x_axis_label='online p_{T} thresh. [GeV]')


In [65]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, ['V11','NM', 'TDR'], 'PU200', ['EG'], 'EGq5', None)
for his in hsets:
    his.normalize(31000)
draw([his.h_pt for his in hsets], labels, text=text,  
     min_y=1, max_y=40000,
     min_x=0, max_x=60,
     logy=True, x_axis_label='online p_{T} thresh. [GeV]')


normalize to 31000
normalize to 31000
normalize to 31000


In [66]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, ['V11','NM'], 'PU200', ['tkEleEllAll'], 'all', None)
for his in hsets:
    his.normalize(31000)
draw([his.h_pt for his in hsets], labels=['before fix', 'after-fix'], text=text,  
     min_y=1, max_y=40000,
     min_x=0, max_x=60,
     logy=True, x_axis_label='online p_{T} thresh. [GeV]')


normalize to 31000
normalize to 31000


In [ ]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, ['V10', 'V11','V9', 'TDR'], 'PU200', ['HMvDR'], 'EmEtaBCD', None)
for his in hsets:
    his.normalize(31000)
draw([his.h_pt for his in hsets], labels, text=text,  
     min_y=1, max_y=40000,
     min_x=0, max_x=60,
     logy=True, x_axis_label='online p_{T} thresh. [GeV]')


In [ ]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, ['TDR'], 'PU200', ['HMvDR', 'HMvDRshapeDrCalib'], 'EmEtaBC', None)
for his in hsets:
    his.normalize(31000)
draw([his.h_pt for his in hsets], labels, text=text,  
     min_y=1, max_y=40000,
     min_x=0, max_x=60,
     logy=True, x_axis_label='online p_{T} thresh. [GeV]')


In [ ]:
tps = ['HMvDRshapeDrCalib', 
       'HMvDRshapeDrCalibNew'
      ]

In [ ]:
for sample in samples:
    for tp in tps:
        for tp_sel in ['EmEtaBC', 'EmEtaBCD', 'EmEtaB', 'EmEtaC']:
            hsets, labels, text = hplot.get_histo(histos.RateHistos, sample.type, 'PU200', tp, tp_sel, None)
#             print hsets
            if hsets is None:
                continue
            for his in hsets:
                his.normalize(31000)
                his.h_pt_graph(
                    'online', 
                    'rate [kHz]', 
                    convertRateToGraph)
                his.h_pt_graph(
                    'offline', 
                    'rate [kHz]', 
                    lambda x: convertRateToGraph(
                        x, 
                        eval(scaling_params[
                                (scaling_params['sample'] == 'ele-'+sample.type) &
                                (scaling_params.pu == 'PU200') & 
                                (scaling_params.tp == tp) & 
                                (scaling_params.tp_sel == tp_sel)].params.iloc[0])))

In [ ]:
hplot.labels_dict.update({'HMvDRshapeDrCalibNew': 'new calib'})

In [ ]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, ['V11', 'TDR'], 'PU200', ['HMvDRshapeDrCalib', 'HMvDRshapeDrCalibNew'], 'EmEtaBC', None)
draw([his.g_pt_online for his in hsets], labels, text=text,  
     min_y=1, max_y=40000,
     min_x=0, max_x=60,
     logy=True, x_axis_label='online p_{T} thresh. [GeV]')


In [ ]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', 'HMvDRshapeDrCalib', 'EmEtaBC', None)
draw([his.g_pt_offline for his in hsets], labels, text=text,  
     min_y =1, max_y=40000,
     min_x=0, max_x=60,
     logy=True, x_axis_label='offline p_{T} thresh. [GeV]')


In [ ]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', 'HMvDRshapeDrCalib', 'EmEtaB', None)
draw([his.g_pt_offline for his in hsets], labels, text=text,  
     min_y =1, max_y=40000,
     min_x=0, max_x=60,
     logy=True, x_axis_label='offline p_{T} thresh. [GeV]')


In [ ]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', 'HMvDRshapeDrCalib', 'EmEtaC', None)
draw([his.g_pt_online for his in hsets], labels, text=text,  
     min_y =1, max_y=40000,
     min_x=0, max_x=60,
     logy=True, x_axis_label='online p_{T} thresh. [GeV]')


In [ ]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', 'HMvDRshapeDrCalib', 'EmEtaC', None)
draw([his.g_pt_offline for his in hsets], labels, text=text,  
     min_y =1, max_y=40000,
     min_x=0, max_x=60,
     logy=True, x_axis_label='offline p_{T} thresh. [GeV]')


In [ ]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', 'HMvDRshapeDrCalib', 'EmEtaBCD', None)
draw([his.g_pt_online for his in hsets], labels, text=text,  
     min_y =1, max_y=40000,
     min_x=0, max_x=60,
     logy=True, x_axis_label='online p_{T} thresh. [GeV]')


In [ ]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', 'HMvDRshapeDrCalib', 'EmEtaBCD', None)
draw([his.g_pt_offline for his in hsets], labels, text=text,  
     min_y =1, max_y=40000,
     min_x=0, max_x=60,
     logy=True, x_axis_label='offline p_{T} thresh. [GeV]')


In [ ]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', 'HMvDRshapeDrCalib', 'EmEtaBC', None)
draw([his.g_pt_offline for his in hsets], labels, text=text,  
     min_y =1, max_y=40000,
     min_x=0, max_x=60,
     logy=True, x_axis_label='offline p_{T} thresh. [GeV]')


In [ ]:
dm = DrawMachine(rleg_config)
hsets, labels, text = hplot.get_histo(histos.RateHistos, 'TDR', 'PU200', 'HMvDRshapeDrCalib', 'EmEtaBCD', None)

dm.addHistos([his.g_pt_offline for his in hsets], ['offline'])
dm.addHistos([his.g_pt_online for his in hsets], ['online'])

dm.draw(
    text=text,
    y_min =1, y_max=40000,
    x_min=0, x_max=60,
    y_log=True, x_axis_label='p_{T} thresh. [GeV]')



In [ ]:
dm = DrawMachine(rleg_config)
hsets, labels, text = hplot.get_histo(histos.RateHistos, 'V9', 'PU200', 'HMvDRshapeDrCalib', 'EmEtaBCD', None)

dm.addHistos([his.g_pt_offline for his in hsets], ['offline'])
dm.addHistos([his.g_pt_online for his in hsets], ['online'])

dm.draw(
    text=text,
    y_min =1, y_max=40000,
    x_min=0, x_max=60,
    y_log=True, x_axis_label='p_{T} thresh. [GeV]')


In [ ]:
dm = DrawMachine(rleg_config)
hsets, labels, text = hplot.get_histo(histos.RateHistos, 'V10', 'PU200', 'HMvDRshapeDrCalib', 'EmEtaBCD', None)

dm.addHistos([his.g_pt_offline for his in hsets], ['offline'])
dm.addHistos([his.g_pt_online for his in hsets], ['online'])

dm.draw(
    text=text,
    y_min =1, y_max=40000,
    x_min=0, x_max=60,
    y_log=True, x_axis_label='p_{T} thresh. [GeV]')


In [ ]:
dm = DrawMachine(rleg_config)
hsets, labels, text = hplot.get_histo(histos.RateHistos, 'V11', 'PU200', 'HMvDRshapeDrCalib', 'EmEtaBCD', None)

dm.addHistos([his.g_pt_offline for his in hsets], ['offline'])
dm.addHistos([his.g_pt_online for his in hsets], ['online'])

dm.draw(
    text=text,
    y_min =1, y_max=40000,
    x_min=0, x_max=60,
    y_log=True, x_axis_label='p_{T} thresh. [GeV]')


In [21]:
scaling_params

,Unnamed: 0,tp,tp_sel,params,sample,pu
0,0,EG,EGq5EtaBC,"(6.037856085779983, 1.0989328915838663)",ele-V9,PU0
1,1,TkEleEL,EGq5EtaBC,"(5.355004263517035, 1.1165288275760397)",ele-V9,PU0
2,2,EGBRL,allEtaF,"(4.33427086849217, 1.0092957507706983)",ele-V9,PU0
3,3,EGBRL,LooseTkIDEtaF,"(3.510329235810837, 1.0305315294454913)",ele-V9,PU0
4,4,TkEleELBRL,allEtaF,"(0.1749948001067856, 1.109074287128954)",ele-V9,PU0
5,5,EG,EGq5EtaBC,"(7.892955246439467, 1.0450598534917357)",ele-V9,PU200
6,6,TkEleEL,EGq5EtaBC,"(5.9020892979972714, 1.0955363006244943)",ele-V9,PU200
7,7,EGBRL,allEtaF,"(4.209194442902704, 0.9991760195740905)",ele-V9,PU200
8,8,EGBRL,LooseTkIDEtaF,"(6.734526207925173, 0.9312246788755905)",ele-V9,PU200
9,9,TkEleELBRL,allEtaF,"(2.5102979469553617, 1.0332268479075164)",ele-V9,PU200


In [22]:
for sample_scaling, sample_rate, tp, tp_sel, tp_scaling, tp_sel_scaling in [
    ('ele-V9', 'TDR', 'EG', 'EGq5EtaBC', 'EG', 'EGq5EtaBC'),
    ('ele-V9', 'TDR', 'EGBRL', 'EtaF', 'EGBRL', 'allEtaF'),
    ('ele-V9', 'TDR', 'TkEleELBRL', 'EtaF', 'TkEleELBRL', 'allEtaF'),
    ('ele-V9', 'TDR', 'TkEleEL', 'EGq5EtaBC', 'TkEleEL', 'EGq5EtaBC'),
    ('ele-V9', 'TDR', 'TkEleELALL', 'Iso0p1EtaBC', 'TkEleEL', 'EGq5EtaBC'),
    ('ele-V9', 'TDR', 'TkEleELALL', 'EtaFIso0p1', 'TkEleELBRL', 'allEtaF'),
    ('dele-NM', 'NM', 'EG', 'EGq5EtaBC', 'EG', 'EGq5EtaBC'),
    ('dele-NM', 'NM', 'EGBRL', 'EtaF', 'EGBRL', 'allEtaF'),
    ('dele-NM', 'NM', 'tkEleEB', 'EtaF', 'tkEleEB', 'allEtaF'),
    ('dele-NM', 'NM', 'tkEleEE', 'EGq5EtaBC', 'tkEleEE', 'EGq5EtaBC'),
    ('ele-NM', 'TDR', 'tkEle', 'Iso0p1EtaBC', 'tkEleEE', 'EGq5EtaBC'),
    ('ele-NM', 'TDR', 'tkEle', 'EtaFIso0p1', 'tkEleEB', 'allEtaF'),

]:
    
        
    hsets, labels, text = hplot.get_histo(histos.RateHistos, sample_rate, 'PU200', tp, tp_sel, None)
    if hsets is None:
        continue
    for his in hsets:
        his.normalize(31000)
        
        his.h_pt_graph(
            'online', 
            'rate [kHz]', 
            convertRateToGraph)
        his.h_pt_graph(
            'offline', 
            'rate [kHz]', 
            lambda x: convertRateToGraph(
                x, 
                eval(scaling_params[
                        (scaling_params['sample'] == sample_scaling) &
                        (scaling_params.pu == 'PU200') & 
                        (scaling_params.tp == tp_scaling) & 
                        (scaling_params.tp_sel == tp_sel_scaling)].params.iloc[0])))

normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
No match found for: sample: TDR pu: PU200, tp: TkEleELALL, tp_sel: Iso0p1EtaBC, gen_sel: None, classtype: python.l1THistos.RateHistos
normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
No match found for: sample: TDR pu: PU200, tp: tkEle, tp_sel: Iso0p1EtaBC, gen_sel: None, classtype: python.l1THistos.RateHistos
No match found for: sample: TDR pu: PU200, tp: tkEle, tp_sel: EtaFIso0p1, gen_sel: None, classtype: python.l1THistos.RateHistos


In [23]:
dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, 'TDR', 'PU200', 'EG', 'EGq5EtaBC', None)
dm.addHistos([his.g_pt_online for his in hsets], [labels[0] + 'EG sta. TDR'])
# dm.addHistos([his.g_pt_online for his in hsets], ['online'])


hsets, labels, text = hplot.get_histo(histos.RateHistos, 'NM', 'PU200', 'EG', 'EGq5EtaBC', None)
dm.addHistos([his.g_pt_online for his in hsets], [labels[0] + 'EG sta. CMSSW11.1'])


hsets, labels, text = hplot.get_histo(histos.RateHistos, 'TDR', 'PU200', 'TkEleEL', 'EGq5EtaBC', None)
dm.addHistos([his.g_pt_online for his in hsets], [labels[0] + 'TkEle TDR'])

hsets, labels, text = hplot.get_histo(histos.RateHistos, 'NM', 'PU200', 'tkEleEE', 'EGq5EtaBC', None)
dm.addHistos([his.g_pt_online for his in hsets], [labels[0] + 'TkEle CMSSW11.1'])


dm.draw(
    text='PU200, EG barrel, 1.52 < |#eta^{L1}| <= 2.4',
    y_min =1, y_max=40000,
    x_min=0, x_max=60,
    y_log=True, x_axis_label='p_{T} online thresh. [GeV]')

dm.write(name='eg_TDRvsSummer20_EE_rate_online')

Info in <TCanvas::Print>: pdf file eg_TDRvsSummer20_EE_rate_online.pdf has been created


In [24]:
dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, 'TDR', 'PU200', 'EG', 'EGq5EtaBC', None)
dm.addHistos([his.g_pt_offline for his in hsets], [labels[0] + 'EG sta. TDR'])
# dm.addHistos([his.g_pt_online for his in hsets], ['online'])


hsets, labels, text = hplot.get_histo(histos.RateHistos, 'NM', 'PU200', 'EG', 'EGq5EtaBC', None)
dm.addHistos([his.g_pt_offline for his in hsets], [labels[0] + 'EG sta. CMSSW11.1'])


hsets, labels, text = hplot.get_histo(histos.RateHistos, 'TDR', 'PU200', 'TkEleEL', 'EGq5EtaBC', None)
dm.addHistos([his.g_pt_offline for his in hsets], [labels[0] + 'TkEle TDR'])

hsets, labels, text = hplot.get_histo(histos.RateHistos, 'NM', 'PU200', 'tkEleEE', 'EGq5EtaBC', None)
dm.addHistos([his.g_pt_offline for his in hsets], [labels[0] + 'TkEle CMSSW11.1'])


dm.draw(
    text='PU200, EG barrel, 1.52 < |#eta^{L1}| <= 2.4',
    y_min =1, y_max=40000,
    x_min=0, x_max=60,
    y_log=True, x_axis_label='p_{T} offline thresh. [GeV]')

dm.write(name='eg_TDRvsSummer20_EE_rate')

Info in <TCanvas::Print>: pdf file eg_TDRvsSummer20_EE_rate.pdf has been created


In [27]:
dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, 'TDR', 'PU200', 'EG', 'EGq5EtaBC', None)
dm.addHistos([his.g_pt_offline for his in hsets], [labels[0] + 'EG sta. TDR'])
# dm.addHistos([his.g_pt_online for his in hsets], ['online'])

hsets, labels, text = hplot.get_histo(histos.RateHistos, 'TDR', 'PU200', 'EG', 'EGq4EtaBC', None)
dm.addHistos([his.g_pt_offline for his in hsets], [labels[0] + 'EG sta. TDR'])
# dm.addHistos([his.g_pt_online for his in hsets], ['online'])


hsets, labels, text = hplot.get_histo(histos.RateHistos, 'NM', 'PU200', 'EG', 'EGq5EtaBC', None)
dm.addHistos([his.g_pt_offline for his in hsets], [labels[0] + 'EG sta. CMSSW11.1'])


hsets, labels, text = hplot.get_histo(histos.RateHistos, 'TDR', 'PU200', 'TkEleEL', 'EGq5EtaBC', None)
dm.addHistos([his.g_pt_offline for his in hsets], [labels[0] + 'TkEle TDR'])

hsets, labels, text = hplot.get_histo(histos.RateHistos, 'NM', 'PU200', 'tkEleEE', 'EGq5EtaBC', None)
dm.addHistos([his.g_pt_offline for his in hsets], [labels[0] + 'TkEle CMSSW11.1'])


dm.draw(
    text='PU200, EG barrel, 1.52 < |#eta^{L1}| <= 2.4',
    y_min =1, y_max=40000,
    x_min=0, x_max=60,
    y_log=True, x_axis_label='p_{T} offline thresh. [GeV]')



AttributeError: RateHistos instance has no attribute 'g_pt_offline'

In [25]:
dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, 'TDR', 'PU200', 'EGBRL', 'EtaF', None)
dm.addHistos([his.g_pt_online for his in hsets], [labels[0] + 'EG sta. TDR'])
# dm.addHistos([his.g_pt_online for his in hsets], ['online'])


hsets, labels, text = hplot.get_histo(histos.RateHistos, 'NM', 'PU200', 'EGBRL', 'EtaF', None)
dm.addHistos([his.g_pt_online for his in hsets], [labels[0] + 'EG sta. CMSSW11.1'])


hsets, labels, text = hplot.get_histo(histos.RateHistos, 'TDR', 'PU200', 'TkEleELBRL', 'EtaF', None)
dm.addHistos([his.g_pt_online for his in hsets], [labels[0] + 'TkEle TDR'])

hsets, labels, text = hplot.get_histo(histos.RateHistos, 'NM', 'PU200', 'tkEleEB', 'EtaF', None)
dm.addHistos([his.g_pt_online for his in hsets], [labels[0] + 'TkEle CMSSW11.1'])


dm.draw(
    text='PU200, EG barrel, |#eta^{L1}| <= 1.479',
    y_min =1, y_max=40000,
    x_min=0, x_max=60,
    y_log=True, x_axis_label='p_{T} online thresh. [GeV]')

# dm.write(name='eg_TDRvsSummer20_EBrate')

In [23]:
dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, 'TDR', 'PU200', 'EGBRL', 'EtaF', None)
dm.addHistos([his.g_pt_offline for his in hsets], [labels[0] + 'EG sta. TDR'])
# dm.addHistos([his.g_pt_online for his in hsets], ['online'])


hsets, labels, text = hplot.get_histo(histos.RateHistos, 'NM', 'PU200', 'EGBRL', 'EtaF', None)
dm.addHistos([his.g_pt_offline for his in hsets], [labels[0] + 'EG sta. CMSSW11.1'])


hsets, labels, text = hplot.get_histo(histos.RateHistos, 'TDR', 'PU200', 'TkEleELBRL', 'EtaF', None)
dm.addHistos([his.g_pt_offline for his in hsets], [labels[0] + 'TkEle TDR'])

hsets, labels, text = hplot.get_histo(histos.RateHistos, 'NM', 'PU200', 'tkEleEB', 'EtaF', None)
dm.addHistos([his.g_pt_offline for his in hsets], [labels[0] + 'TkEle CMSSW11.1'])


dm.draw(
    text='PU200, EG barrel, |#eta^{L1}| <= 1.479',
    y_min =1, y_max=40000,
    x_min=0, x_max=60,
    y_log=True, x_axis_label='p_{T} offline thresh. [GeV]')

dm.write(name='eg_TDRvsSummer20_EBrate')

Info in <TCanvas::Print>: pdf file eg_TDRvsSummer20_EBrate.pdf has been created
